# Linear Regression

In [ ]:
# Import
import pandas as pd
import numpy as np

# Load data
dataset = pd.read_csv("../dataset/student-mat.csv", delimiter=";")

# Select important data
important_columns = ["age", "Medu", "Fedu", "traveltime", 
                    "studytime", "failures", "famrel",
                     "freetime", "goout", "Dalc", "Walc",
                    "health", "absences"]
X = dataset.loc[:,important_columns].values
Y = dataset.loc[:,"G3"].values

# Prepare datasets
dataset_length = len(X)
cv_start_index = int(0.6*dataset_length)
test_start_index = int(0.8*dataset_length)

X_training = X[:cv_start_index]
Y_training = Y[:cv_start_index]
X_cv = X[cv_start_index:test_start_index]
Y_cv = Y[cv_start_index:test_start_index]
X_test = X[test_start_index:]
Y_test = Y[test_start_index:]

In [ ]:
# Hypothesis function
def h(THETA, X):
    return X.dot(THETA)

In [ ]:
# Cost function
def J(THETA, X, Y):
    return 1/(2*len(X)) * np.sum((h(THETA, X)-Y)**2)

# Derrivative of cost function respects to i-th THETA parameter
def dJ(THETA, X, Y, i):
    return 1/(len(X)) * np.sum((h(THETA, X)-Y)*X[:,i])

In [ ]:
# Gradient Descent
def gradient_descent(X, Y, a):
    THETA = np.zeros(X.shape[1])
    while True:
        NEW_THETA = np.copy(THETA)
        for i in range(len(THETA)):
            NEW_THETA[i] = THETA[i] - a*dJ(THETA, X, Y, i)
#         print(J(NEW_THETA, X, Y))
        print(J(THETA, X, Y))
        if J(NEW_THETA, X, Y) <= J(THETA, X, Y):
            THETA = NEW_THETA
        else:
            return THETA

In [ ]:
t = gradient_descent(X_training, Y_training, 0.001)

In [ ]:
print(t)